In [1]:
# temporary hack to avoid a very complex bug related to jupyter, ipykernel, asyncio, snakemake
import nest_asyncio
nest_asyncio.apply()

# Demo of the Periodic Hill case

The [phill example](https://github.com/KTH-Nek5000/KTH_Examples/tree/master/phill_STAT) has been adapted for a workflow using snek5000 and pymech. Here we will show how a workflow looks like. To get started, we install everything we need:

```sh
python -m venv venv
source venv/bin/activate
pip install snek5000 pymech -e "git+https://github.com/exabl/snek5000-phill#egg=phill"
```

## Initialize and setup up simulation paramters

In [2]:
from phill.solver import Simul

params = Simul.create_default_params()

The `params` object gives you a consolidated view of the parameters which are spread out in a typical Nek5000 case into `.par`, `.box` and `SIZE` file.  Already we seen that the parameters are more verbose, easier to understand. As a bonus, some parameters which depend on others are automatically set. For example, see {py:mod}`snek5000.operators`.

Now let us take a look at all the compilation parameters that we can modify. In a console the params would also output as follows:

In [3]:
print(params)


<params NEW_DIR_RESULTS="True" short_name_type_run="run">
  <oper Lx="1.0" Ly="1.0" Lz="1.0" boundary="['P', 'P', 'W', 'W', 'P', 'P']"
        boundary_scalars="[]" dim="3" nproc_max="32" nproc_min="8" nx="22"
        ny="16" nz="19" origin_x="0.0" origin_y="0.0" origin_z="0.0"
        ratio_x="1.0" ratio_y="1.0" ratio_z="1.0" scalars="1">
    <max dim_krylov="30" dim_proj="20" hist="1" obj="1" perturb="1"
         scalars_cons="1" scalars_proj="1" sessions="1"/>  

    <elem coef_dealiasing="0.6666666666666666" order="6" order_out="6"
          staggered="True"/>  

    <misc fast_diag="False"/>  

  </oper>

  <output HAS_TO_SAVE="True" ONLINE_PLOT_OK="True" period_refresh_plots="1"
          sub_directory=""/>  

  <nek>
    <general dealiasing="True" dt="-0.0002" end_time="nan"
             extrapolation="standard" filter_cutoff_ratio="0.67"
             filter_modes="2" filter_weight="0.02" filtering="explicit"
             log_level="2" num_steps="20" opt_level="2" start_from=""

One can print some help about some parameters. For example, for `params.oper`:

In [4]:
params.oper._print_docs()

Documentation for params.oper
-----------------------------

Parameters for mesh description:

- ``nx``, ``ny``, ``nz``: int
    Number of elements in each directions
- ``origin_x``, ``origin_y, ``origin_z``: float
    Starting coordinate of the mesh (default: 0.0)
- ``ratio_x``, ``ratio_y``, ``ratio_z``: float
    Mesh stretching ratio (default: 1.0)
- ``Lx``, ``Ly``, ``Lz``: float
    Length of the domain

Parameters for boundary conditions:

- ``boundary``: list[str]
    `Velocity boundary conditions <https://nek5000.github.io/NekDoc/problem_setup/boundary_conditions.html#fluid-velocity>`__
- ``boundary_scalars``: list[str]
    `Temperature and passive scalar boundary conditions <https://nek5000.github.io/NekDoc/problem_setup/boundary_conditions.html#temperature-and-passive-scalars>`__

The following table matches counterpart of mandatory ``SIZE`` variables.

==========  ===================   =============================================
SIZE        params.oper           Comment
===

The parameters can be modified. For instance, let us tweak the number of elements, time-stepping and I/O parameters

In [5]:
# This affects both the box and SIZE files
params.oper.nx = 12
params.oper.ny = 10
params.oper.nz = 8

params.oper.nproc_min = 2

# This affects the par file 
params.nek.general.num_steps = 10
params.nek.general.time_stepper = "bdf2"
params.nek.general.write_interval = 10

params.nek.stat.av_step = 3
params.nek.stat.io_step = 10

Now initialize the simulation. This would copy the files based on the templates we have specified.

In [6]:
sim = Simul(params)
sim.path_run

No module named 'paraview'
******************************************
solver: <class 'phill.output.OutputPhill'>
path_run: /home/pierre/Sim_data/phill_run_12x10x8_V1.x1.x1._2021-09-17_00-02-29
******************************************
Searching for a log file...
Cannot find a .log to parse in /home/pierre/Sim_data/phill_run_12x10x8_V1.x1.x1._2021-09-17_00-02-29.
Writing params files... /home/pierre/Sim_data/phill_run_12x10x8_V1.x1.x1._2021-09-17_00-02-29/phill.par, params_simul.xml, info_solver.xml
Copying with shutil.copytree ...
Copied: /home/pierre/Dev/snek5000-phill/src/phill -> /home/pierre/Sim_data/phill_run_12x10x8_V1.x1.x1._2021-09-17_00-02-29
Writing box file... /home/pierre/Sim_data/phill_run_12x10x8_V1.x1.x1._2021-09-17_00-02-29/phill.box
Writing SIZE file... /home/pierre/Sim_data/phill_run_12x10x8_V1.x1.x1._2021-09-17_00-02-29/SIZE


sim:                <class 'phill.solver.SimulPhill'>
sim.output:         <class 'phill.output.OutputPhill'>


PosixPath('/home/pierre/Sim_data/phill_run_12x10x8_V1.x1.x1._2021-09-17_00-02-29')

In [7]:
!ls {sim.path_run}

SIZE       etc              makefile_usr.inc  phill.box  phill.par  templates
Snakefile  info_solver.xml  params_simul.xml  phill.log  phill.usr  toolbox


To run the simulation we need to execute certain commands. These are described using snakemake in the Snakefile. Let's look at the rules defined in the Snakefile (which are nearly generic for any Nek5000 case).

In [8]:
sim.make.list()

Changing to shadow directory: /home/pierre/Sim_data/phill_run_12x10x8_V1.x1.x1._2021-09-17_00-02-29
Using os.cpu_count() to detect maximum number of processors available


all
mesh
show_config
build_third_party
compile
mpiexec
run
run_fg
clean
cleansimul
clean3rd
cleanall
archive
internal_generate_box
internal_move_box
internal_generate_map
internal_generate_makefile
internal_generate_session


True

The rules in the Snakefile are either shell commands or Python code which handle different parts of the build step, such as building a mesh (rule `mesh`), compiling (rule `compile`) and running the simulation (rule `run` or `run_fg`). The rules can be executed on by one by passing them as strings to the `exec` method of the `sim.make` object. The default parameter is to do everyting to run a simulation.

In [9]:
sim.make.exec?

Signature: sim.make.exec(rules=('run',), dryrun=False, **kwargs)
Docstring:
Execute snakemake rules in sequence.

:param iterable rules: Snakemake rules to be executed
:param bool dryrun: Dry run snakemake without executing

For more on available keyword arguments refer to `Snakemake API documentation`_.

:returns: True if workflow execution was successful.

.. _Snakemake API documentation: https://snakemake.readthedocs.io/en/stable/api_reference/snakemake.html

Examples
--------

>>> sim.make.exec(['compile'])
>>> sim.make.exec(['run'], resources={'nproc': 4})

It is also possible to do the same directly from command line
by changing to the simulation directory and executing::

    snakemake -j compile
    snakemake -j1 --resources nproc=2 run

The flag ``-j`` is short for ``--jobs`` and sets the number of
threads available for snakemake rules to execute.

.. seealso:: Useful Snakemake `command line arguments`_

.. _command line arguments: https://snakemake.readthedocs.io/en/stable/ex

In [10]:
sim.make.exec()

Changing to shadow directory: /home/pierre/Sim_data/phill_run_12x10x8_V1.x1.x1._2021-09-17_00-02-29


Using os.cpu_count() to detect maximum number of processors available
Building DAG of jobs...
Executing subworkflow Nek5000.
Building DAG of jobs...
Nothing to be done.
Complete log: /home/pierre/Dev/Nek5000/.snakemake/log/2021-09-17T000230.604952.snakemake.log
Executing main workflow.
Using shell: /bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job                           count    min threads    max threads
--------------------------  -------  -------------  -------------
compile                           1              1              1
internal_generate_box             1              1              1
internal_generate_makefile        1              1              1
internal_generate_map             1              1              1
internal_generate_session         1              1              1
internal_move_box                 1              1              1
run                               1             

 input file name:
-3                                         spatial dimension (dim<0 will create box.re2)
2                                          number of fields
 T T F           2   2.0000000000000000     
           1  i1#
           1  i1#
           1  i1#
           1  i1#
           1  i1#
           1  i1#
           1  i1#
           1  i1#
           1  i1#
           1  i1#
           1  i1#
           1  i1#
           1  i1#
           1  i1#
           1  i1#
           1  i1#
Box
-12 -10 -8                                nelx nely nelz
Reading         960 =       12       10        8 elements for box   1.
Reading           2 =
0.0000 1.0000 1.0000                      x0 x1 ratio
0.0000 1.0000 1.0000                      y0 y1 ratio
0.0000 1.0000 1.0000                      z0 z1 ratio
P  ,P  ,W  ,W  ,P  ,P                     Velocity BCs
 Beginning construction of box.re2
         960  elements will be created for            1  boxes.

Input .rea / .re2 name:
 read

[Fri Sep 17 00:02:30 2021]
Finished job 3.
4 of 7 steps (57%) done
Select jobs to execute...

[Fri Sep 17 00:02:30 2021]
rule internal_generate_makefile:
    input: phill.re2, phill.ma2, phill.usr, /home/pierre/Dev/Nek5000/bin/nekconfig
    output: makefile, .state
    jobid: 6
    resources: tmpdir=/tmp



 done:    0.9% 
 done:    1.9% 
 done:    2.8% 
 done:    3.8% 
 done:    4.7% 
 done:    5.6% 
 done:    6.6% 
 done:    7.5% 
 done:    8.4% 
 done:    9.4% 
 done:   10.3% 
 done:   11.2% 
 done:   12.2% 
 done:   13.1% 
 done:   14.1% 
 done:   15.0% 
 done:   15.9% 
 done:   16.9% 
 done:   17.8% 
 done:   18.8% 
 done:   19.7% 
 done:   20.6% 
 done:   21.6% 
 done:   22.5% 
 done:   23.4% 
 done:   24.4% 
 done:   25.3% 
 done:   26.2% 
 done:   27.2% 
 done:   28.1% 
 done:   29.1% 
 done:   30.0% 
 done:   30.9% 
 done:   31.9% 
 done:   32.8% 
 done:   33.8% 
 done:   34.7% 
 done:   35.6% 
 done:   36.6% 
 done:   37.5% 
 done:   38.4% 
 done:   39.4% 
 done:   40.3% 
 done:   41.2% 
 done:   42.2% 
 done:   43.1% 
 done:   44.1% 
 done:   45.0% 
 done:   45.9% 
 done:   46.9% 
 done:   47.8% 
 done:   48.8% 
 done:   49.7% 
 done:   50.6% 
 done:   51.6% 
 done:   52.5% 
 done:   53.4% 
 done:   54.4% 
 done:   55.3% 
 done:   56.2% 
 done:   57.2% 
 done:   58.1% 
 done:  

...

/home/pierre/Dev/Nek5000/core/3rd_party/iniparser.c: In function ‘iniparser_load’:
/home/pierre/Dev/Nek5000/core/3rd_party/iniparser.c:718:32: warning: ‘__builtin___sprintf_chk’ may write a terminating nul past the end of the destination [-Wformat-overflow=]
  718 |             sprintf(tmp, "%s:%s", section, key);
      |                                ^
In file included from /usr/include/stdio.h:867,
                 from /home/pierre/Dev/Nek5000/core/3rd_party/iniparser.h:16,
                 from /home/pierre/Dev/Nek5000/core/3rd_party/iniparser.c:11:
/usr/include/x86_64-linux-gnu/bits/stdio2.h:36:10: note: ‘__builtin___sprintf_chk’ output between 2 and 2050 bytes into a destination of size 2049
   36 |   return __builtin___sprintf_chk (__s, __USE_FORTIFY_LEVEL - 1,
      |          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
   37 |       __bos (__s), __fmt, __va_arg_pack ());
      |       ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


mpif77  -O2 -cpp -fdefault-real-8 -fdefault-double-8 -w -std=legacy -march=native -mcmodel=medium -std=legacy -Itoolbox -DMPI -DUNDERSCORE -DGLOBAL_LONG_LONG -DTIMER -I/home/pierre/Sim_data/phill_run_12x10x8_V1.x1.x1._2021-09-17_00-02-29 -I/home/pierre/Dev/Nek5000/core -I./ -I /home/pierre/Dev/Nek5000/core/experimental  -c /home/pierre/Sim_data/phill_run_12x10x8_V1.x1.x1._2021-09-17_00-02-29/phill.f -o obj/phill.o


ar: `u' modifier ignored since `D' is the default (see `U')
[Fri Sep 17 00:02:40 2021]
Finished job 5.
6 of 7 steps (86%) done
Select jobs to execute...

[Fri Sep 17 00:02:40 2021]
rule run:
    input: phill.re2, phill.ma2, phill.par, SESSION.NAME, nek5000
    log: logs/run_2021-09-17T00-02-30.log
    jobid: 0
    resources: tmpdir=/tmp, nproc=6

[Fri Sep 17 00:02:40 2021]
Finished job 0.
7 of 7 steps (100%) done
Complete log: /home/pierre/Dev/Nek5000/.snakemake/log/2021-09-17T000230.604952.snakemake.log


done
mpif77 -c  -O2 -cpp -fdefault-real-8 -fdefault-double-8 -w -std=legacy -march=native -mcmodel=medium -std=legacy -Itoolbox -DMPI -DUNDERSCORE -DGLOBAL_LONG_LONG -DTIMER -I/home/pierre/Sim_data/phill_run_12x10x8_V1.x1.x1._2021-09-17_00-02-29 -I/home/pierre/Dev/Nek5000/core -I./ -I /home/pierre/Dev/Nek5000/core/experimental /home/pierre/Dev/Nek5000/core/drive.f
mpif77  -O2 -cpp -fdefault-real-8 -fdefault-double-8 -w -std=legacy -march=native -mcmodel=medium -std=legacy -Itoolbox -DMPI -DUNDERSCORE -DGLOBAL_LONG_LONG -DTIMER -I/home/pierre/Sim_data/phill_run_12x10x8_V1.x1.x1._2021-09-17_00-02-29 -I/home/pierre/Dev/Nek5000/core -I./ -I /home/pierre/Dev/Nek5000/core/experimental -o nek5000 drive.o obj/phill.o obj/libnek5000.a -L/home/pierre/Dev/Nek5000/3rd_party/blasLapack -lblasLapack -L/home/pierre/Dev/Nek5000/3rd_party/gslib/lib -lgs -Wl,--allow-multiple-definition


#############################################################
#                  Compilation successful!             

True

In [11]:
!ls {sim.path_run}

SESSION.NAME  etc              makefile_usr.inc  phill.box  phill.re2
SIZE          info_solver.xml  nek5000           phill.log  phill.usr
Snakefile     logs             obj               phill.ma2  templates
build.log     makefile         params_simul.xml  phill.par  toolbox


The simulation is done!